In [1]:
# just to have it for google colab
# !pip install z3-solver

In [2]:
from z3 import *
import time
import math

DA FARE:
1. ~~Create benchmark suite~~
2. Documenta + pulisci il codice
3. Creare delle funzioni per i modelli che condividono gli stessi constraints
6. Ultime idee del file doc ((PB) literature search (VALE PUNTI!) e ~~encoding lineare~~)
7. Guarda quali sono le cose che danno punti extra -> CONVERTING THE SOLVER OBJECT TO DIMACS AND PLAY WITH DIFFERENT SAT SOLVERS (This can solve the encoding time problem! But on the other hand it can't be done for the incremental model! Fai entrambi cosi prendi punti extra ma fai vedere la soluzione migliore)
8. Implementa l'ultima idea di modello (ricerca in t_compressed con colonne di t ordinate in base ad a) (Penso sia finito e funzionante, controlla)
10. Fai testing di performance di tutte le combinazioni! Questo poi quando scrivi il template
11. Implementa la binary search per il modello incrementale
12. prova il modello incrementale con 3 solver! Trovo a, poi trovo t e poi uso gli ultimi constraint solo per calcolare le distanze rispettive al t trovato. Prova sia con i constraints su righe e colonne di r che senza nei "terzi" constraints, a vedere se cambia
13. prova il modello incrementale (con 2 o 3 solver in base a cosa ha funzionato meglio prima) anche sul modello "big path": prima trova t_compressed, poi orders + implied a per i loads (segmento di ogni corriere, ordinati, e rispettivi constraint dei pesi) e poi r per calcolare le distanze
14. PROVA t->r model ma con constraint su r solo gli all_false!! I.e. ultimo caso possibile di bijective

TODO! non ho messo AllLessEq(upper_bound) con l'iniziale upper_bound per la linear search! Grave...ma in realtà forse non ne ho bisogno no? Perchè è implicita nel problema, anzi la ho già imposta con l'implicit constraint

## Utils

In [3]:
def millisecs_left(t, timeout):
    """returns the amount of milliseconds left from t to timeout

    Args:
        t (int): timestamp
        timeout (int): timestamp of the timeout

    Returns:
        int: the milliseconds left
    """
    return int((timeout - t) * 1000)

In [4]:
def flatten(matrix):
    """flattens a 2D list into a 1D list

    Args:
        matrix (list[list[Object]]): the matrix to flatten

    Returns:
        list[Object]: the flattened 1D list
    """
    return [e for row in matrix for e in row]

#### Conversion from int to bin

In [5]:
def num_bits(x):
    """Returns the number of bits necessary to represent the integer x

    Args:
        x (int): the input integer

    Returns:
        int: the number of bits necessary to represent x
    """
    return math.floor(math.log2(x)) + 1

In [6]:
def int_to_bin(x, digits):
    """Converts an integer x into a binary representation of True/False using #bits = digits

    Args:
        x (int): the integer to convert
        digits (int): the number of bits of the output

    Returns:
        list[bool]: the binary representation of x
    """
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

def bin_to_int(x):
    """Converts a binary number of 1s and 0s into its integer form

    Args:
        x (list[int]): the binary number to convert

    Returns:
        int: the converted integer representation of x
    """
    n = len(x)
    x = sum(2**(n-i-1) for i in range(n) if x[i] == 1)
    return x

## Encodings

#### At most/least one & exactly one

In [7]:
def at_least_one(bool_vars):
    """Z3 encoding of "At least one" over bool_vars

    Args:
        bool_vars (list[Bool]): the list of input Z3 Bool objects 

    Returns:
        Z3-expression: the Z3 encoding of "At least one" over bool_vars
    """
    return Or(bool_vars)

def at_most_one_seq(x, name):
    """Z3 encoding of "At most one" using sequential encoding

    Args:
        x (list[Bool]): the list of input Z3 Bool objects
        name (str): the name to append to the auxiliary variables

    Returns:
        Z3-expression: the Z3 encoding of "At most one" over x
    """
    n = len(x)
    if n == 1:
        return True
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # s[i] modeled as: s[i] is true iff the sum up to index i is 1

    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0]
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    """Z3 encoding of "Exactly one" using sequential encoding

    Args:
        bool_vars (list[Bool]): the list of input Z3 Bool objects
        name (str): the name to append to the auxiliary variables

    Returns:
        Z3-expression: the Z3 encoding of "Exactly one" over x
    """
    return And(at_least_one(bool_vars), at_most_one_seq(bool_vars, name))

#### Operations on binary numbers

In [8]:
def equal(v, u):
    """Z3 encoding of "Equal" position-wise

    Args:
        v (list[Bool]): the first term
        u (list[Bool]): the second term

    Returns:
        Z3-expression: the Z3 encoding of "Equal"
    """
    assert(len(v) == len(u))
    return And([v[k]==u[k] for k in range(len(v))])

def all_false(v):
    """Z3 encoding of "All false"

    Args:
        v (list[Bool]): the input list of Bools

    Returns:
        Z3-expression: the Z3 encoding of "All false"
    """
    return And([Not(v[k]) for k in range(len(v))])

def leq_same_digits(v, u, digits):
    """Encoding of v <= u, implementation with digits fixed and equal between v and u

    Args:
        v (list[Bool]): binary representation of v with Z3 Bool variables
        u (list[Bool]): binary representation of u with Z3 Bool variables
        digits (int): number of digits of v and u

    Returns:
        (Z3-expression): encoding of v <= u in binary considering their {digits} most significant bits
    """
    assert(len(v) == len(u) and len(u) == digits)
    if digits == 1:
        return Or(v[0]==u[0], And(Not(v[0]), u[0]))
    else:
        return Or(And(Not(v[0]), u[0]),
                  And(v[0]==u[0], leq_same_digits(v[1:], u[1:], digits-1)))


def leq(v, u):
    """Encoding of v <= u, implementation with different digits btw v and u

    Args:
        v (list[Bool]): binary representation of v with Z3 Bool variables
        u (list[Bool]): binary representation of u with Z3 Bool variables

    Returns:
        (Z3-expression): encoding of v <= u in binary
    """
    digits_v = len(v)
    digits_u = len(u)

    if digits_v == digits_u:
        return leq_same_digits(v, u, digits_v)
    elif digits_v < digits_u:
        delta_digits = digits_u - digits_v
        return Or(Or(u[:delta_digits]),
                  leq_same_digits(v, u[delta_digits:], digits_v))
    else:
        delta_digits = digits_v - digits_u
        return And(all_false(v[:delta_digits]), leq_same_digits(v[delta_digits:], u, digits_u))


In [9]:
def sum_bin_same_digits(a_bin, b_bin, d_bin, digits, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[Bool]): binary representation of a with Z3 Bool variables
        b_bin (list[Bool]): binary representation of b with Z3 Bool variables
        d_bin (list[Bool]): binary representation of d with Z3 Bool variables
        digits (int): number of bits of each number
        name (str): string to identify carry boolean variables

    Returns:
        formula (Z3-expression): formula representing SAT encoding of binary sum
        c[0] (Bool): last carry of binary encoding
    """
    # c_k represents carry at bit position k
    c = [Bool(f"c_{k}_{name}") for k in range(digits+1)]
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    formula = And(clauses)
    return (formula, c[0])

def sum_bin(a_bin, b_bin, d_bin, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, with digits(a_bin) <= digits(b_bin) == digits(d_bin)

    Args:
        a_bin (list[Bool]): binary representation of a with Z3 Bool variables
        b_bin (list[Bool]): binary representation of b with Z3 Bool variables
        d_bin (list[Bool]): binary representation of d with Z3 Bool variables
        name (str): string to identify carry boolean variables

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """
    digits_a = len(a_bin)
    digits_b = len(b_bin)
    digits_d = len(d_bin)
    assert(digits_a <= digits_b and digits_b == digits_d)

    delta_digits = digits_b - digits_a

    if delta_digits == 0:
        formula, last_carry = sum_bin_same_digits(a_bin, b_bin, d_bin, digits_a, name)
        return And(formula, Not(last_carry))    # imposing no overflow

    sub_sum_formula, last_carry = sum_bin_same_digits(a_bin, b_bin[delta_digits:], d_bin[delta_digits:], digits_a, name)
    c = [Bool(f"c_propagated_{k}_{name}") for k in range(delta_digits)] + [last_carry]
    c[0] = False     # imposing no further overflow

    clauses = []
    for k in range(delta_digits-1, -1, -1):
        clauses.append(d_bin[k] == Xor(b_bin[k], c[k+1]))
        clauses.append(c[k] == And(b_bin[k], c[k+1]))

    return And(And(clauses), sub_sum_formula)

In [10]:
def conditional_sum_K_bin(x, alpha, delta, name):
    """Encodes into a SAT formula the constraint {delta = sum_over_j(alpha[j] | x[j] == True)}

    Args:
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        delta (list[Bool]): list of Z3 Variables, which will be constrained to represent the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint

    """
    n = len(x)
    digits = len(delta)

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n-1)]    # j = 1..n-1 because last row will be delta
    d.append(delta)

    clauses = []

    # row 0
    diff_digits = digits - len(alpha[0])
    assert(diff_digits >= 0)
    clauses.append( And(Implies(x[0], And(all_false(d[0][:diff_digits]), equal(d[0][diff_digits:], alpha[0]))), # If x[0] == 1 then d_0 == alpha_0 (with eventual padding of zeros)
                        Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]

    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(x[j], sum_bin(alpha[j], d[j-1], d[j], f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j
                            Implies(Not(x[j]), equal(d[j], d[j-1]))))

    return And(clauses)

#### Linear Integer constraint over binary numbers

In [11]:
# TODO: it's not used anymore now, can remove it now! ...or better, don't remove it and implement PB on it to optimize it
# Then find a way to also make it return the result of the sum as a Z3-Variable so that can impose symmetry breaking constraint on it!
def LinearInteger_bin(x, alpha, beta, name):
    """Encodes into a SAT formula the linear constraint {sum_over_j(alpha[j] | x[j] == True) <= beta}

    Args:
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        beta (list[bool]): binary representing the known term, i.e. the upper bound for the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    n = len(x)
    digits = len(beta)

    clauses = []


    sum_result = [Bool(f"LIsum_result_{k}_{name}") for k in range(digits)]
    clauses.append(conditional_sum_K_bin(x, alpha, sum_result, name))

    clauses.append(leq(sum_result, beta))

    formula = And(clauses)
    return formula



#### Distances travelled and objective function encodings

In [12]:
# TODO: non viene usato! Ho ricopiato il codice in tutti gli usi. Usalo solo se poi fai la cosa delle tante funzioni che fanno gli encoding
def distances_def_constraint(distances, flat_r, flat_D_bin):
    """Defines the set of binary numbers {distances} as the sum of the binary numbers in {flat_D_bin} whose respective element in {flat_r} is True
        i.e. distances[i] = sum_over_j(flat_D_bin[j] | (flat_r[i])[j] == True)  forall i

    Args:
        distances (list[list[Bool]]): distances[i] will be constrained to be the sum of flat_D_bin[j] with respective flat_r[i][j] == True
        flat_r (list[list[Bool]]]): list of flattened matrices of Z3 Bool variables, each matrix flat_r[i] originally (i.e. before flattening) describing the route of courier i
        flat_D_bin (list[list[bool]]): flattened matrix of distances D, with each element converted to a binary as a list[bool]

    Returns:
        (Z3-expression): encoding of the definition of distances
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    return And(clauses)


def obj_function(model, distances):
    """Given a model, returns the objective function value that we are interested in (i.e. max of distances) as an integer

    Args:
        model (ModelRef): model of which to compute the objective function
        distances (list[list[Bool]]): list containing the binary representation (using Z3 Bool variables) of each distance

    Returns:
        int: the maximum distance travelled
    """
    m = len(distances)
    maxdist = -1
    for i in range(m):
        dist = bin_to_int([1 if model.evaluate(distances[i][j]) else 0 for j in range(len(distances[i]))])
        maxdist = max(maxdist, dist)
    return maxdist

def AllLessEq_bin(distances, upper_bound_bin):
    """Encodes the constraint {Forall i. distances[i] <= upper_bound_bin}

    Args:
        distances (list[list[Bool]]): list containing the binary representation (using Z3 Bool variables) of each distance
        upper_bound_bin (list[bool]): binary representation of the upper bound

    Returns:
        Z3-Expression: the constraint encoding
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(distances[i], upper_bound_bin))

    return And(clauses)

def AtLeastOneGreaterEq_bin(distances, lower_bound_bin):
    """Encodes the constraint {Exists i. distances[i] >= lower_bound_bin}

    Args:
        distances (list[list[Bool]]): list containing the binary representation (using Z3 Bool variables) of each distance
        lower_bound_bin (list[bool]): binary representation of the lower bound

    Returns:
        Z3-Expression: the constraint encoding
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(lower_bound_bin, distances[i]))

    return Or(clauses)

## Functions to display solution

In [13]:
def displayMCP(orders, distances_bin, obj_value, assignments=None):
    distances = [bin_to_int(d) for d in distances_bin]

    print(f"-----------Objective value: {obj_value}-----------")
    print(f"------------------Routes-----------------")
    if assignments is None:
        m = len(orders)
        n = len(orders[0])
        for i in range(m):
            visited = [n]
            for t in range(n):
                found = False
                for j in range(n):
                    if orders[i][j][t]:
                        visited.append(j)
                        found = True
                if not found:
                    break
            visited.append(n)

            print('Origin --> ' +
                  ' --> '.join([str(vertex) for vertex in visited[1:-1]]) +
                  f' --> Origin: travelled {distances[i]}')

    else:
        m = len(assignments)
        n = len(assignments[0])
        t = 0
        i = 0
        visited = [n]
        while t < n:
            for j in range(n):
                if orders[j][t]:
                    if assignments[i][j]:
                        visited.append(j)
                    else:
                        visited.append(n)
                        print('Origin --> ' + ' --> '.join(
                            [str(vertex) for vertex in visited[1:-1]]) +
                              f' --> Origin: travelled {distances[i]}')
                        visited = [n, j]
                        i += 1
                    break
            t += 1
        visited.append(n)
        print('Origin --> ' +
              ' --> '.join([str(vertex) for vertex in visited[1:-1]]) +
              f' --> Origin: travelled {distances[i]}')


#### Functions to evaluate values of matrices or tensors

In [1]:
def evaluate(model, bools):
    """Evaluate every element of bools using model recursively

    Args:
        model (ModelRef): the model to evaluate on
        bools (n-dim list[Bool]): the bools to evaluate, can be of arbitrary dimension

    Returns:
        n-dim list[int]: object of the same dimensions of bools, with a 1 in the corresponding position of 
                         the bools that evaluated to true w.r.t. model
    """
    if not isinstance(bools[0], list):
        return [1 if model.evaluate(b) else 0 for b in bools]
    return [evaluate(model, b) for b in bools]

#### Function to check that routes are all Hamiltonian cycles

In [15]:
def check_all_hamiltonian(tensor):
    """Function to check that all the paths represented in tensor are hamiltonian cycles

    Args:
        tensor (list[list[list[bool]]]): list of adjacency matrices over 2-regular graphs

    Returns:
        bool: true iff all paths in tensor are hamiltonian cycles, false otherwise
    """
    m = len(tensor)
    for i in range(m):
        if not check_hamiltonian(tensor[i]):
            return False
    return True

def check_hamiltonian(matrix):
    """Function to check that the given adjancency matrix over 2-regular graph is a hamiltonian cycle, i.e. the graph is connected

    Args:
        matrix (list[list[bool]]): adjacency matrix over 2-regular graph

    Returns:
        bool: true iff the given adjancency matrix is a hamiltonian cycle, false otherwise
    """
    n = len(matrix)
    visited = [0] * n
    v = n-1
    while visited[v] == 0:
        visited[v] = 1
        for k in range(n):
            if matrix[v][k] == True:
                v = k
                break
    num_vertices = sum(row.count(True) for row in matrix)
    return (sum(visited) == num_vertices)


## Implicit constraints

Assumpions that we can make:
1. $n >= m$
2. The distance matrix $D$ is quasimetric (more info [here](https://proofwiki.org/wiki/Definition:Quasimetric)) => Giving an item to courier who has none is always "same or better" than giving 2 to another courier and none to this one

The assumptions above give us the implicit constraint **"Every courier has at least one item assigned to it"**

## Symmetry breaking constraints

TODO: spiega symmetry breaking constraints

In [16]:
def sort_decreasing(matrix):
    """Encoding of the constraint that the binary numbers represented by the rows of {matrix} are sorted in decreasing order

    Args:
        matrix (list[list[Bool]]): matrix[i] represents an integer in binary using Z3 Bool variables

    Returns:
        (Z3-expression): constraint representing the decreasing ordering of binary numbers in the rows of matrix
    """
    m = len(matrix)
    clauses = []
    for i in range(m-1):
        clauses.append(leq(matrix[i+1], matrix[i]))
    return And(clauses)


In [ ]:
# TODO: metti qui le funzioni di ogni constraint (se lo fai alla fine)

## Models

* Model 1: r and t matrices, optional symmetry breaking on loads
* Model 2: r, r_compressed and t_compressed matrices (s.t. to reduce encoding time), optional symmetry breaking on both loads and t_compressed

All implement both Linear and Binary Optimization search

In [17]:
# TODO: remove spiegando nel report la funzione di ricorrenza che scala peggio
def multiple_couriers_planning_r_to_t(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        # TODO: count this towards solving time by subtracting its time to the timeout
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")

    # Constraint 4: routes

    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin

        # Constraint 4.4: HAMILTONIAN CYCLES
        # Uses matrix t[i] in order to check that the adjacency matrix r[i] represents a Hamiltonian cycle
        for j in range(n):
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            # 1st trip
            solver.add(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) --> j) => t_ij0 (i delivers j as 1st item)

            # successive trips
            # r_ijk (courier i goes from j to k) iff (t_ijh AND t_ik{h+1} for some h) (i delivers j at time h and k at time h+1, for some h)
            for k in range(n):
                solver.add(r[i][j][k] == Or([And(t[i][j][h], t[i][k][h+1]) for h in range(0, n-1)]))


    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # Complexity: O(n) NICE!
    # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration


    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))


    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model, t)
        Dists = evaluate(model, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)


In [20]:
def multiple_couriers_planning_t_to_r(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4: routes

    for i in range(m):
        ## Constraints on r
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))


        ## Constraints on t
        for j in range(n):
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

        for j in range(n-1):
            # when starts false -> all false afterwards
            solver.add(Implies(all_false([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])))

        ## channeling constraints
        for j in range(n):

            # 1st trip
            solver.add(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) <--> j) => t_ij0 (i delivers j as 1st item)

            for k in range(n):

                solver.add(Or([And(t[i][j][h-1], t[i][k][h]) for h in range(1, n)]) == r[i][j][k])

        for j in range(n-1):
            # Find the last object delivered and place 1 in returning edge
            solver.add(And(at_least_one([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])) == equal([t[i][k][j] for k in range(n)], [r[i][k][n] for k in range(n)]))






    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # TODO: remove
    # if now >= timeout:
    #     return "Encoding time too slow"

    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))


    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model, t)
        Dists = evaluate(model, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)

In [19]:
def multiple_couriers_planning_bijective(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4: routes

    for i in range(m):
        ## Constraints on r
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin
        ### end TODO

        # Constraints on t
        for j in range(n):
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

        for j in range(n-1):
            # when starts false -> all false afterwards
            solver.add(Implies(all_false([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])))

        # channeling constraints
        for j in range(n):

            # 1st trip
            solver.add(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) <--> j) => t_ij0 (i delivers j as 1st item)

            for k in range(n):

                solver.add(Or([And(t[i][j][h-1], t[i][k][h]) for h in range(1, n)]) == r[i][j][k])

        for j in range(n-1):
            # Find the last object delivered and place 1 in returning edge
            solver.add(And(at_least_one([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])) == equal([t[i][k][j] for k in range(n)], [r[i][k][n] for k in range(n)]))






    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # TODO: remove
    # if now >= timeout:
    #     return "Encoding time too slow"

    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))


    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model, t)
        Dists = evaluate(model, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)


In [18]:
def multiple_couriers_planning_compressed(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 2 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or now to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    r_compressed = [[Bool(f"r_compressed_{j}_{k}") for k in range(n+1)] for j in range(n+1)]
    # r_compressed_jk = 1 indicates that some courier moves from delivery point j to delivery point k


    t_compressed = [[Bool(f"delivers_{j}_as_{h}-th") for h in range(n+1)] for j in range(n)]
    # t_compressed_jh = 1 iff object j is delivered as h-th element (i.e. at TIME=h, use of time as order) considering that
    # couriers leave one at a time and only once the previous one has returned to the origin point
    # last column of t_compressed is imposed to have all 0s in order to guarantee no cycles that exclude the origin


    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m-1):
            solver.add(Implies(equal(courier_loads[i], courier_loads[i+1]), leq(a[i], a[i+1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")




    # Constraint 4: routes
    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin

    # TODO: remove
    print(f"Finised constraint 4 at time {round(time.time() - start_time, 1)}s")


    # Constraint 4.4: HAMILTONIAN CYCLES
    # Derive matrix r_compressed from tensor r by "compressing" it over the couriers-dimension (using Or operator)
    # Derive matrix t_compressed from r_compressed

    # r_compressed def
    for j in range(n+1):
        for k in range(n+1):
            solver.add(Implies(Or([r[i][j][k] for i in range(m)]), r_compressed[j][k]))
            solver.add(Implies(all_false([r[i][j][k] for i in range(m)]), Not(r_compressed[j][k])))

    # t_compressed def
    for j in range(n):
        solver.add(exactly_one_seq(t_compressed[j], f'no_double_delivery_{j}'))
        solver.add(exactly_one_seq([t_compressed[h][j] for h in range(n)], f'no_contemporary_delivery_{j}'))
    solver.add(all_false([t_compressed[h][n] for h in range(n)]))

    # 1st trip
    solver.add(equal(r[0][n][:n], [t_compressed[j][0] for j in range(n)]))  # start from deliveries of 1st courier in t_compressed

    # successive trips
    for h in range(1, n+1):
        for j in range(n):
            # (Exists k s.t. i came from that k to j AND he delivered k at time h-1) ==> (j delivered at time h)
            for k in range(n):
                solver.add(Implies(And(r_compressed[k][j], t_compressed[k][h-1]), t_compressed[j][h]))  # r_i,k,j AND t_i,k,h-1 (i goes from k to j AND he delivered k at time h-1) => t_i,j,h (j delivered at time h)

            # Don't include the inverse implication (<==) because won't be satisfied for deliveries of two different couriers

    # TODO: remove
    print(f"Finised constraint Hamiltonian (r_compressed and t_compressed def) at time {round(time.time() - start_time, 1)}s")


    # Constraint 4.4.1: symmetry breaking constraint over the definition of t_compressed
    if symmetry_breaking:
        # couriers_orders_ki = 1 iff object delivered at time k is delivered by courier i
        couriers_orders = [[Bool(f"courier_{i}_delivers_{k}-th_object") for i in range(m)] for k in range(n)]

        for k in range(n):
            solver.add(exactly_one_seq(couriers_orders[k], f'only_one_courier_delivers_at_time_{k}'))

        for i in range(m):
            for j in range(n):
                for k in range(n):
                    # (object j assigned to i AND object j is delivered at time k)
                    # ==> object delivered at time k is delivered by courier i
                    solver.add(Implies(And(a[i][j], t_compressed[j][k]), couriers_orders[k][i]))

        # put in t_compressed first all objects delivered by courier 0, then 1, then 2, ...
        for k in range(n-1):
            for i in range(m-1):
                # next object is either still of courier i or of courier i+1
                solver.add(Implies(couriers_orders[k][i], Xor(couriers_orders[k+1][i], couriers_orders[k+1][i+1])))

                # direct search: if there is a change in couriers, start (in t_compressed) from 1st object delivered by next courier
                solver.add(Implies(And(couriers_orders[k][i], couriers_orders[k+1][i+1]), equal([t_compressed[j][k+1] for j in range(n)], r[i+1][n][:n])))
            # if reached last courier, all next elements are delivered by him
            solver.add(Implies(couriers_orders[k][m-1], couriers_orders[k+1][m-1]))

    # TODO: remove
    print(f"Finished constraint 'Symmetry breaking on t_compressed' at time {round(time.time() - start_time, 1)}s")



    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: represent as Bool binary number the distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])     # TODO: maybe we can find a tighter upper bound on obj func
    # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # Complexity: O(n) NICE!
    # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    ## VARIABLE
    # distances_i = representation in binary of the distance travelled by courier i
    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    ## CONSTRAINT
    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished constraint 'distances_def' at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration

    # TODO: remove
    # if now >= timeout:
    #     return "Encoding time too slow"

    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))

            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))

    # TODO: try to skew the search towards lower values, like by not dividing by 2 every time but like doing 1/4 from lower_bound, or sqrt of difference
    # (...but it's really theoretically wrong because switching point (minimum) is really uniformly distributed!)
    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T_compressed = evaluate(model, t_compressed)
        A = evaluate(model, a)
        Dists = evaluate(model, distances)
        displayMCP(T_compressed, Dists, obj_value, A)

    return (obj_value, solving_time)


In [21]:
def multiple_couriers_planning_incremental(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)


    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]

    # Bounds on objective function
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])


    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    
    def assignments_constraints():
        clauses = []
        
        ## CONSTRAINTS
        if symmetry_breaking:
            # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
            clauses.append(sort_decreasing(courier_loads))
            # Break symmetry within same load amounts, i.e.:
            # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
            # i.e. the second courier will be the one assigned to the route containing the item with lower index j
            for i in range(m - 1):
                clauses.append(
                    Implies(equal(courier_loads[i], courier_loads[i + 1]),
                            leq(a[i], a[i + 1])))
                
        # Constraint 1: every object is assigned to one and only one courier
        for j in range(n):
            clauses.append(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
                
        # Constraint 2: every courier can't exceed its load capacity
        for i in range(m):
            clauses.append(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
            clauses.append(leq(courier_loads[i], l_bin[i]))

        # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
        for i in range(m):
            clauses.append(at_least_one(a[i]))

        return And(clauses)


    def routes_constraints():
        clauses = []

        # Constraint 4: routes
        for i in range(m):
            ## Constraints on r
            # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
            clauses.append(And([Not(r[i][j][j]) for j in range(n+1)]))


            ## Constraints on t
            for j in range(n):
                # assignments implications
                clauses.append(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
                clauses.append(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

                # can't deliver two items at the same time
                clauses.append(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            for j in range(n-1):
                # when starts false -> all false afterwards
                clauses.append(Implies(all_false([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])))

            ## channeling constraints
            for j in range(n):

                # 1st trip
                clauses.append(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) <--> j) => t_ij0 (i delivers j as 1st item)

                for k in range(n):

                    clauses.append(Or([And(t[i][j][h-1], t[i][k][h]) for h in range(1, n)]) == r[i][j][k])

            for j in range(n-1):
                # Find the last object delivered and place 1 in returning edge
                clauses.append(And(at_least_one([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])) == equal([t[i][k][j] for k in range(n)], [r[i][k][n] for k in range(n)]))

        # definition of distances using constraints
        for i in range(m):
            clauses.append(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

        return And(clauses)



    ## OPTIMIZATION SEARCH

    model_sub = None
    model_master = None
    obj_value = None
    exit_flag = False



    solver_sub = Solver()
    solver_master = Solver()

    sub_constraints = assignments_constraints()
    solver_sub.add(sub_constraints)
    solver_master.add(sub_constraints)

    master_constraints = routes_constraints()
    solver_master.add(master_constraints)

    encoding_time = time.time()
    timeout = encoding_time + timeout_duration
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")


    if search == 'Linear':

        solver_master.push()

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver_master.add(AllLessEq_bin(distances, upper_bound_bin))

        solver_sub.set('timeout', millisecs_left(time.time(), timeout))
        while solver_sub.check() == z3.sat and not exit_flag:
            print(f"Found a valid A after {round(time.time() - encoding_time, 1)}s")

            model_sub = solver_sub.model()

            solver_master.push()

            # impose the found assignments on the master problem
            for i in range(m):
                for j in range(n):
                    solver_master.add(a[i][j] == model_sub.evaluate(a[i][j]))

            solver_master.push()

            now = time.time() 
            if now >= timeout:
                break
            solver_master.set('timeout', millisecs_left(now, timeout))
            while solver_master.check() == z3.sat:

                model_master = solver_master.model()

                obj_value = obj_function(model_master, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= lower_bound:
                    exit_flag = True
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


                solver_master.pop()
                solver_master.push()

                solver_master.add(AllLessEq_bin(distances, upper_bound_bin))

                now = time.time()
                if now >= timeout:
                    exit_flag = True
                    break
                solver_master.set('timeout', millisecs_left(now, timeout))

            solver_master.pop()     # remove the latest found distance constraint frame
            solver_master.pop()     # remove the assignments constraint frame
            solver_master.pop()     # remove the past holding distance constraint frame

            solver_master.push()    # re-create the distance constraint frame
            solver_master.add(AllLessEq_bin(distances, upper_bound_bin))


            # force at least one difference in the assignments matrix 'a' w.r.t the last matrix of assignments found
            solver_sub.add(Or([Not(a[i][j]) if model_sub.evaluate(a[i][j]) else a[i][j] for i in range(m) for j in range(n)]))
            
            now = time.time()
            if now >= timeout:
                break
            solver_sub.set('timeout', millisecs_left(now, timeout))
   

    # TODO: binary search for incremental
    elif search == 'Binary':

        if solver_sub.check() == z3.sat:
            
            solver_master.add(sub_constraints)
            solver_master.add(routes_constraints())



        
        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model_master is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model_master, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model_master, t)
        Dists = evaluate(model_master, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)

In [ ]:
def multiple_couriers_planning_incremental_v2(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)


    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]

    # Bounds on objective function
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])


    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]


    def create_assignment_constraints():
        clauses = []

        ## CONSTRAINTS
        if symmetry_breaking:
            # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
            clauses.append(sort_decreasing(courier_loads))
            # Break symmetry within same load amounts, i.e.:
            # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
            # i.e. the second courier will be the one assigned to the route containing the item with lower index j
            for i in range(m - 1):
                clauses.append(
                    Implies(equal(courier_loads[i], courier_loads[i + 1]),
                            leq(a[i], a[i + 1])))

        # Constraint 1: every object is assigned to one and only one courier
        for j in range(n):
            clauses.append(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

        # Constraint 2: every courier can't exceed its load capacity
        for i in range(m):
            clauses.append(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
            clauses.append(leq(courier_loads[i], l_bin[i]))

        # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
        for i in range(m):
            clauses.append(at_least_one(a[i]))

        return And(clauses)


    def create_routes_constraints():
        clauses = []

        # Constraint 4: routes
        for i in range(m):
            ## Constraints on r
            # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
            clauses.append(And([Not(r[i][j][j]) for j in range(n+1)]))


            ## Constraints on t
            for j in range(n):
                # assignments implications
                clauses.append(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
                clauses.append(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

                # can't deliver two items at the same time
                clauses.append(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            for j in range(n-1):
                # when starts false -> all false afterwards
                clauses.append(Implies(all_false([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])))

        return And(clauses)


    def create_distances_constraints():
        clauses = []

        ## r constraints
        for j in range(n):

            # 1st trip
            clauses.append(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) <--> j) => t_ij0 (i delivers j as 1st item)

            for k in range(n):

                clauses.append(Or([And(t[i][j][h-1], t[i][k][h]) for h in range(1, n)]) == r[i][j][k])

        for j in range(n-1):
            # Find the last object delivered and place 1 in returning edge
            clauses.append(And(at_least_one([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])) == equal([t[i][k][j] for k in range(n)], [r[i][k][n] for k in range(n)]))


        # definition of distances using constraints
        for i in range(m):
            clauses.append(
                conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i],
                                      f"distances_def_{i}"))
            
        return And(clauses)




    ## OPTIMIZATION SEARCH

    model_assignments = None
    model_routes = None
    model_distances = None
    obj_value = None
    exit_flag = False



    solver_assignments = Solver()
    solver_routes = Solver()
    solver_distances = Solver() # TODO: al solver_distances posso non passare 'a' giusto!? Devo farlo solo se uso il constraint sulle righe e colonne di a

    assignment_constraints = create_assignment_constraints()
    solver_assignments.add(assignment_constraints)
    solver_routes.add(assignment_constraints)

    routes_constraints = create_routes_constraints()
    solver_routes.add(routes_constraints)
    solver_distances.add(routes_constraints)

    distances_constraints = create_distances_constraints()
    solver_distances.add(distances_constraints)

    upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
    upper_bound_constraint = AllLessEq_bin(distances, upper_bound_bin))

    encoding_time = time.time()
    timeout = encoding_time + timeout_duration
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")


    if search == 'Linear':

        # SOLVE FOR ASSIGNMENTS
        solver_assignments.set('timeout', millisecs_left(time.time(), timeout))
        while solver_assignments.check() == z3.sat and not exit_flag:
            print(f"Found a valid A after {round(time.time() - encoding_time, 1)}s")
            
            # retrieve the found assignments and impose them on the routes problem
            model_assignments = solver_assignments.model()

            solver_routes.push()

            for i in range(m):
                for j in range(n):
                    solver_routes.add(a[i][j] == model_assignments.evaluate(a[i][j]))

            now = time.time()
            if now >= timeout:
                break
            solver_routes.set('timeout', millisecs_left(now, timeout))
            while solver_routes.check() == z3.sat and not exit_flag:
                print(f"Found a valid t after {round(time.time() - encoding_time, 1)}s")

                # retrieve the found assignments and impose them on the distances problem
                model_routes = solver_routes.model()

                solver_distances.push()

                for j in range(n):
                    for k in range(n):
                        solver_distances.add(t[j][k] == model_routes.evaluate(t[j][k]))

                # impose the last found upper bound
                solver_distances.push()
                solver_distances.add(upper_bound_constraint)

                now = time.time()
                if now >= timeout:
                    break
                solver_distances.set('timeout', millisecs_left(now, timeout))
                while solver_distances.check() == z3.sat:
                    
                    model_distances = solver_distances.model()
                    
                    obj_value = obj_function(model_routes, distances)
                    print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                    upper_bound = obj_value - 1
                    upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
                    upper_bound_constraint = AllLessEq_bin(distances, upper_bound_bin)

                    if obj_value <= lower_bound:
                        exit_flag = True
                        break

                    solver_distances.pop()
                    solver_distances.push()
                    solver_distances.add(upper_bound_constraint)

                    now = time.time()
                    if now >= timeout:
                        exit_flag = True
                        break
                    solver_distances.set('timeout', millisecs_left(now, timeout))

                # remove the t and the upper_bound constraints
                solver_distances.pop()
                solver_distances.pop()

                # force at least one difference in the routes matrix 't' w.r.t. the last routes matrix found
                solver_routes.add(Or([Not(t[i][j]) if model_assignments.evaluate(t[i][j]) else t[i][j] for i in range(n) for j in range(n)]))


                now = time.time()
                if now >= timeout:
                    exit_flag = True
                    break
                solver_routes.set('timeout', millisecs_left(now, timeout))

            solver_routes.pop()     # remove the assignments constraint frame


            # force at least one difference in the assignments matrix 'a' w.r.t the last matrix of assignments found
            solver_assignments.add(Or([Not(a[i][j]) if model_assignments.evaluate(a[i][j]) else a[i][j] for i in range(m) for j in range(n)]))

            now = time.time()
            if now >= timeout:
                break
            solver_assignments.set('timeout', millisecs_left(now, timeout))


    # TODO: binary search for incremental
    elif search == 'Binary':

        if solver_assignments.check() == z3.sat:

            solver_routes.add(assignment_constraints)
            solver_routes.add(create_routes_constraints())




        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model_distances is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model_distances, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model_distances, t) 
        Dists = evaluate(model_distances, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)

In [22]:
def multiple_couriers_planning_incremental_push_pop(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars

    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)]
    # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i

    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    if symmetry_breaking:
        # sort the list of loads
        l.sort(reverse=True)


    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]

    # Bounds on objective function
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])


    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    
    def assignments_constraints():
        clauses = []
        
        ## CONSTRAINTS
        if symmetry_breaking:
            # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
            clauses.append(sort_decreasing(courier_loads))
            # Break symmetry within same load amounts, i.e.:
            # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
            # i.e. the second courier will be the one assigned to the route containing the item with lower index j
            for i in range(m - 1):
                clauses.append(
                    Implies(equal(courier_loads[i], courier_loads[i + 1]),
                            leq(a[i], a[i + 1])))
                
        # Constraint 1: every object is assigned to one and only one courier
        for j in range(n):
            clauses.append(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
                
        # Constraint 2: every courier can't exceed its load capacity
        for i in range(m):
            clauses.append(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
            clauses.append(leq(courier_loads[i], l_bin[i]))

        # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
        for i in range(m):
            clauses.append(at_least_one(a[i]))

        return And(clauses)


    def routes_constraints():
        clauses = []

        # Constraint 4: routes
        for i in range(m):
            ## Constraints on r
            # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
            clauses.append(And([Not(r[i][j][j]) for j in range(n+1)]))


            ## Constraints on t
            for j in range(n):
                # assignments implications
                clauses.append(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) => [t_ijk = True, for some k] (i delivers j at some time k)
                clauses.append(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) => [t_ijk = False, forall k] (i never delivers j)

                # can't deliver two items at the same time
                clauses.append(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            for j in range(n-1):
                # when starts false -> all false afterwards
                clauses.append(Implies(all_false([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])))

            ## channeling constraints
            for j in range(n):

                # 1st trip
                clauses.append(r[i][n][j] == t[i][j][0]) # r_inj (i rides n (origin) <--> j) => t_ij0 (i delivers j as 1st item)

                for k in range(n):

                    clauses.append(Or([And(t[i][j][h-1], t[i][k][h]) for h in range(1, n)]) == r[i][j][k])

            for j in range(n-1):
                # Find the last object delivered and place 1 in returning edge
                clauses.append(And(at_least_one([t[i][k][j] for k in range(n)]), all_false([t[i][k][j+1] for k in range(n)])) == equal([t[i][k][j] for k in range(n)], [r[i][k][n] for k in range(n)]))

        # definition of distances using constraints
        for i in range(m):
            clauses.append(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

        return And(clauses)



    ## OPTIMIZATION SEARCH

    model_sub = None
    model_master = None
    obj_value = None
    exit_flag = False


    solver = Solver()
    solver.add(assignments_constraints())
    
    routes_constrs = routes_constraints()

    encoding_time = time.time()
    timeout = encoding_time + timeout_duration
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")


    if search == 'Linear':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        upper_bound_constr = AllLessEq_bin(distances, upper_bound_bin)

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat and not exit_flag:
            print(f"Found a valid A after {round(time.time() - encoding_time, 1)}s")


            model = solver.model()


            solver.push()
            # impose the found assignments
            for i in range(m):
                for j in range(n):
                    solver.add(a[i][j] == model.evaluate(a[i][j]))

            # TODO: try to add t first and then r and dists
            solver.push()
            solver.add(routes_constrs)

            solver.push()
            solver.add(upper_bound_constr)

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            while solver.check() == z3.sat:

                model = solver.model()

                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= lower_bound:
                    exit_flag = True
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
                upper_bound_constr = AllLessEq_bin(distances, upper_bound_bin)


                solver.pop()    # pop last upper_bound_constr
                solver.push()   # add the new one

                solver.add(upper_bound_constr)

                now = time.time()
                if now >= timeout:
                    exit_flag = True
                    break
                solver.set('timeout', millisecs_left(now, timeout))

            solver.pop()     # remove the latest found upper_bound_constr frame
            solver.pop()     # remove the routes frame
            solver.pop()     # remove the assignments constraints frame

            # force at least one difference in the assignments matrix 'a' w.r.t the last matrix of assignments found
            solver.add(Or([Not(a[i][j]) if model.evaluate(a[i][j]) else a[i][j] for i in range(m) for j in range(n)]))
            
            if now >= timeout:
                exit_flag = True
                break
            solver.set('timeout', millisecs_left(now, timeout))
   


    elif search == 'Binary':

        if solver_sub.check() == z3.sat:
            
            solver_master.add(sub_constraints)
            solver_master.add(routes_constraints())



        
        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T = evaluate(model, t)
        Dists = evaluate(model, distances)
        displayMCP(T, Dists, obj_value)

    return (obj_value, solving_time)

In [23]:
def multiple_couriers_planning_big_path(m, n, l, s, D, symmetry_breaking=True, search='Binary', display_solution=True, timeout_duration=300):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
        symmetry_breaking (bool, optional): wether or not to use symmetry breaking constraints (default=True)
        search (str, optional) ['Linear', 'Binary']: the search strategy to use in the Optimization phase of solving (default='Binary')
        display_solution (bool, optional): wether or not to print the final solution obtained, with the path travelled by each courier (default=True)
        timeout_duration (int, optional): timeout in seconds (default=300)

    """
    start_time = time.time()

    ## VARIABLES

    t_compressed = [[Bool(f"object_{j}_delivered_as_{k}-th") for k in range(n)] for j in range(n)]

    orders = [[Bool(f"courier_{i}_delivers_{k}-th_object") for k in range(n)] for i in range(m)]

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column
    # O(m * n^2) vars


    courier_loads = [[Bool(f"cl_{i}_{k}") for k in range(num_bits(sum(s)))] for i in range(m)]
    # courier_loads_i = binary representation of actual load carried by each courier

    solver = Solver()


    ## CONSTRAINTS
    if symmetry_breaking:
        # sort the list of loads
        # TODO: count this towards solving time by subtracting its time to the timeout
        l.sort(reverse=True)
        # Symmetry breaking constraint 1 -> after having sorted l above, impose the actually couriers_loads to be sorted decreasingly as well
        solver.add(sort_decreasing(courier_loads))
        # Break symmetry within same load amounts, i.e.:
        # if two couriers carry the same load amount, impose a lexicografic ordering on the respective rows of a,
        # i.e. the second courier will be the one assigned to the route containing the item with lower index j
        for i in range(m - 1):
            solver.add(
                Implies(equal(courier_loads[i], courier_loads[i + 1]),
                        leq(a[i], a[i + 1])))

    # Conversions:
    s_bin = [int_to_bin(s_j, num_bits(s_j)) for s_j in s]
    l_bin = [int_to_bin(l_i, num_bits(l_i)) for l_i in l]



    # t_compressed's definition
    for j in range(n):
        solver.add(exactly_one_seq(t_compressed[j], f'object_{j}_delivered'))
        solver.add(exactly_one_seq([t_compressed[k][j] for k in range(n)], f'delivery_at_time_{j}'))

    # orders's definition
    for i in range(m):
        solver.add(at_least_one(orders[i]))
        for j in range(n-1):
            if i < m-1:
                solver.add(Implies(orders[i][j], Xor(orders[i][j+1], orders[i+1][j+1])))
            else:
                solver.add(Implies(orders[i][j], orders[i][j+1]))

    for j in range(n):
        solver.add(exactly_one_seq([orders[i][j] for i in range(m)], f'time_{j}_covered'))

    # implied constraint
    solver.add(orders[0][0])
    solver.add(orders[m-1][n-1])

    # definition of r from t_compressed
    # first courier delivers first item
    solver.add(equal(r[0][n][:n], [t_compressed[j][0] for j in range(n)]))
    # last courier returns from last item
    solver.add(equal([r[m-1][j][n] for j in range(n)], [t_compressed[j][n-1] for j in range(n)]))

    for i in range(m):
        for j in range(n):
            for k in range(n):
                for h in range(n-1):
                    solver.add(Implies(And(t_compressed[j][h], t_compressed[k][h+1], orders[i][h], orders[i][h+1]), r[i][j][k]))

                    if i < m-1:
                        solver.add(Implies(And(t_compressed[j][h], t_compressed[k][h+1], orders[i][h], orders[i+1][h+1]), And(r[i][j][n], r[i+1][n][k])))

                solver.add(Implies(r[i][j][k], Or([And(t_compressed[j][h], t_compressed[k][h+1], orders[i][h], orders[i][h+1]) for h in range(n-1)])))
                if i < m-1:
                    solver.add(Implies(And(r[i][j][n], r[i+1][n][k]), Or([And(t_compressed[j][h], t_compressed[k][h+1], orders[i][h], orders[i+1][h+1]) for h in range(n-1)])))

                

    # definition of a from t_compressed
    for i in range(m):
        for j in range(n):
            # ==>
            for h in range(n):
                solver.add(Implies(And(t_compressed[j][h], orders[i][h]), a[i][j]))

            # <==
            solver.add(Implies(a[i][j], Or([And(t_compressed[j][h], orders[i][h]) for h in range(n)])))

    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))

    # TODO: remove
    print(f"Finished constraint 1 at time {round(time.time() - start_time, 1)}s")


    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(conditional_sum_K_bin(a[i], s_bin, courier_loads[i], f"compute_courier_load_{i}"))
        solver.add(leq(courier_loads[i], l_bin[i]))

    # TODO: remove
    print(f"Finished constraint 2 at time {round(time.time() - start_time, 1)}s")


    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    for i in range(m):
        solver.add(at_least_one(a[i]))

    # TODO: remove
    print(f"Finished constraint 3 at time {round(time.time() - start_time, 1)}s")

    # Constraint 4: routes

    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # TODO: check if better without
        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        # for j in range(n):
        #     solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
        #     solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        # solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # # columns
        # for k in range(n):
        #     solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
        #     solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k)
        # solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin



    # TODO: remove
    print(f"Finished constraint 4 + Hamiltonian at time {round(time.time() - start_time, 1)}s")


    ## OPTIMIZATION SEARCH

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]


    # Constraint 5: distances travelled by each courier
    # distances[i] := binary representation of the distance travelled by courier i
    # Take as upper bound the greater n-(m-1) maximum distances, since that's the maximum items a single courier can be assigned to
    max_distances = [max(D[i]) for i in range(n+1)]
    max_distances.sort()
    upper_bound = sum(max_distances[m-1:])
    # TODO: maybe we can find a tighter upper bound on obj func, i.e. start from a naive solution, that is:
    # starting from the courier with lowest load, assign to him as many objects as possible (starting from the one with lowest load
    # going to the heaviest one, i.e. sort s first! BEWARE, NEED TO KEEP ALSO INITIAL INDEX IN ORDER TO USE PROPER INDEX IN D MATRIX) When he can't take any more object: continue with next courier etc. -> end up with a possible assignment
    # Then compute distances (can do also while assign objects to couriers! All in a single pass) and find the maximum.
    # Complexity: O(n) NICE!
    # Question: ask if you can do this! Pro: would make binary search start way lower! Also could add the initial constraint AllLessEq(distances, upper_bound) because right now doing so it's useless
    # TODO: Do as line above, add constriant AllLessEq(distances, upper_bound)
    # IMP: note that such a solution would NOT be possible if the ACTUAL LOADS ARE NOT SORTED! TODO: find an initial solution that satisfies this!
    lower_bound = max([D[n][j] + D[j][n] for j in range(n)])

    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(num_bits(upper_bound))] for i in range(m)]

    # definition of distances using constraints
    for i in range(m):
        solver.add(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    # TODO: remove
    print(f"Finished distances def constraint at time {round(time.time() - start_time, 1)}s")


    model = None
    obj_value = None
    encoding_time = time.time()
    print(f"Encoding finished at time {round(encoding_time - start_time, 1)}s, now start solving/optimization search")

    timeout = encoding_time + timeout_duration


    solver.push()

    if search == 'Linear':

        solver.set('timeout', millisecs_left(time.time(), timeout))
        while solver.check() == z3.sat:

            model = solver.model()
            obj_value = obj_function(model, distances)
            print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")


            if obj_value <= lower_bound:
                break

            upper_bound = obj_value - 1
            upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            solver.pop()
            solver.push()

            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))


    elif search == 'Binary':

        upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))
        solver.add(AllLessEq_bin(distances, upper_bound_bin))

        lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))
        solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

        while lower_bound <= upper_bound:
            mid = int((lower_bound + upper_bound)/2)
            mid_bin = int_to_bin(mid, num_bits(mid))
            solver.add(AllLessEq_bin(distances, mid_bin))

            now = time.time()
            if now >= timeout:
                break
            solver.set('timeout', millisecs_left(now, timeout))
            print(f"Trying with bounds: [{lower_bound}, {upper_bound}] and posing obj_val <= {mid}")

            if solver.check() == z3.sat:
                model = solver.model()
                obj_value = obj_function(model, distances)
                print(f"This model obtained objective value: {obj_value} after {round(time.time() - encoding_time, 1)}s")

                if obj_value <= 1:
                    break

                upper_bound = obj_value - 1
                upper_bound_bin = int_to_bin(upper_bound, num_bits(upper_bound))


            else:
                print(f"This model failed after {round(time.time() - encoding_time, 1)}s")

                lower_bound = mid + 1
                lower_bound_bin = int_to_bin(lower_bound, num_bits(lower_bound))

            solver.pop()
            solver.push()
            solver.add(AllLessEq_bin(distances, upper_bound_bin))
            solver.add(AtLeastOneGreaterEq_bin(distances, lower_bound_bin))

    else:
        raise ValueError(f"Input parameter [search] mush be either 'Linear' or 'Binary', was given '{search}'")


    # compute time taken
    end_time = time.time()
    if end_time > timeout:
        solving_time = f"{timeout_duration}s -- TIMEOUT"    # solving_time has upper bound of timeout_duration if it timeouts
    else:
        solving_time = f"{round(end_time - encoding_time, 1)}s"

    # if no model is found -> UNSAT
    if model is None:
        return ("UNSAT", solving_time)

    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    assert(check_all_hamiltonian(R))

    if display_solution:
        T_compressed = evaluate(model, t_compressed)
        A = evaluate(model, a)
        Dists = evaluate(model, distances)
        displayMCP(T_compressed, Dists, obj_value, A)

    return (obj_value, solving_time)


### Testing zone

In [24]:
import os
import sys
import numpy as np

def run_model_on_instance(MCP_model, file, **kwargs):
    """Read the instance from .dat file and run the given MCP model on it

    Args:
        MCP_model (function): function executing the SAT-encoding and solving of the given instance
        file (str): path of the .dat file representing the instance
    """
    with open(file) as f:
        m = int(next(f))
        n = int(next(f))
        l = [int(e) for e in next(f).split()]
        s = [int(e) for e in next(f).split()]
        D = np.genfromtxt(f, dtype=int).tolist()

    return MCP_model(m, n, l, s, D, **kwargs)

# TODO: remove num_instances as a param, it's too specific (maybe instances folder changes etc.)
def test_model(MCP_model, output_file, num_instances, **kwargs):
    old_stdout = sys.stdout
    with open(output_file, 'w') as f:
        # TODO: remove stdout redirection, just needed to print partial solutions as well
        sys.stdout = f
        # TODO: make more general with path etc.
        for file in os.listdir('../instances/')[:num_instances]:
            print(f"======================{file}======================", file=f)
            print(f"{file} -> {run_model_on_instance(MCP_model, f'../instances/{file}', **kwargs)}\n", file=f)
            print(f"Finito {file}", file=old_stdout)
        sys.stdout = old_stdout

def compare_models(MCP_modelA, MCP_modelB, modelA_name, modelB_name, num_instances, **kwargs):
    old_stdout = sys.stdout
    with open(f'{modelA_name}_vs_{modelB_name}', 'w') as f:
        # TODO: remove stdout redirection, just needed to print partial solutions as well
        sys.stdout = f
        # TODO: make more general with path etc.
        for file in os.listdir('../instances/')[:num_instances]:
            print(f"======================{file}======================", file=f)

            print(f"----------------------{modelA_name}---------------------", file=f)
            print(f"{file} -> {run_model_on_instance(MCP_modelA, f'../instances/{file}', **kwargs)}\n", file=f)

            print(f"----------------------{modelB_name}---------------------", file=f)
            print(f"{file} -> {run_model_on_instance(MCP_modelB, f'../instances/{file}', **kwargs)}\n", file=f)

            print(f"Finito {file}", file=old_stdout)
        sys.stdout = old_stdout


In [25]:
def compare_list_of_models(models, instances_files, **kwargs):
    for instance_file in instances_files:
        name = instance_file.split('/')[-1]
        print(f'----------{name}-----------')
        for i, model in enumerate(models):
            old_stdout = sys.stdout
            sys.stdout = open(os.devnull, 'w')
            t1 = time.time()
            try:
                ans = run_model_on_instance(model, instance_file, **kwargs)
            except:
                pass
            finally:
                t2 = time.time()
                sys.stdout = old_stdout
            print(f'Model {i}: {ans}, total time: {round(t2 - t1, 1)}s')

In [51]:
run_model_on_instance(multiple_couriers_planning_incremental, '../instances/inst13.dat', search='Linear', symmetry_breaking=False)

Encoding finished at time 7.5s, now start solving/optimization search
Found a valid A after 0.1s
This model obtained objective value: 399 after 0.4s
Found a valid A after 0.4s
Found a valid A after 0.6s
Found a valid A after 0.8s
Found a valid A after 1.2s
Found a valid A after 1.6s
Found a valid A after 1.9s
Found a valid A after 2.2s
This model obtained objective value: 290 after 2.6s
Found a valid A after 2.7s
Found a valid A after 3.0s
Found a valid A after 3.3s
Found a valid A after 3.6s
Found a valid A after 3.9s
Found a valid A after 4.3s
Found a valid A after 4.7s
Found a valid A after 5.0s
Found a valid A after 5.3s
Found a valid A after 5.6s
Found a valid A after 6.0s
Found a valid A after 6.4s
Found a valid A after 6.9s
Found a valid A after 7.2s
Found a valid A after 7.4s
Found a valid A after 7.7s
Found a valid A after 7.9s
Found a valid A after 8.2s
Found a valid A after 8.5s
Found a valid A after 8.8s
Found a valid A after 9.0s
Found a valid A after 9.3s
Found a valid A 

(226, '11.6s')

In [ ]:
run_model_on_instance(multiple_couriers_planning_incremental, '../instances/inst13.dat', search='Linear', symmetry_breaking=True)

Encoding finished at time 7.5s, now start solving/optimization search
Found a valid A after 0.1s
This model obtained objective value: 399 after 0.4s
Found a valid A after 0.4s
Found a valid A after 0.6s
Found a valid A after 0.8s
Found a valid A after 1.2s
Found a valid A after 1.6s
Found a valid A after 1.9s
Found a valid A after 2.2s
This model obtained objective value: 290 after 2.6s
Found a valid A after 2.7s
Found a valid A after 3.0s
Found a valid A after 3.3s
Found a valid A after 3.6s
Found a valid A after 3.9s
Found a valid A after 4.3s
Found a valid A after 4.7s
Found a valid A after 5.0s
Found a valid A after 5.3s
Found a valid A after 5.6s
Found a valid A after 6.0s
Found a valid A after 6.4s
Found a valid A after 6.9s
Found a valid A after 7.2s
Found a valid A after 7.4s
Found a valid A after 7.7s
Found a valid A after 7.9s
Found a valid A after 8.2s
Found a valid A after 8.5s
Found a valid A after 8.8s
Found a valid A after 9.0s
Found a valid A after 9.3s
Found a valid A 

(226, '11.6s')

In [53]:
models = [multiple_couriers_planning_r_to_t, multiple_couriers_planning_t_to_r, multiple_couriers_planning_incremental, multiple_couriers_planning_incremental_2]

compare_list_of_models(models, [f'../instances/inst0{i}.dat' for i in range(7, 10)], search='Linear')

----------inst07.dat-----------
Model 0: (207, '300s -- TIMEOUT'), total time: 336.5s
Model 1: (184, '300s -- TIMEOUT'), total time: 331.8s


In [26]:
models = [multiple_couriers_planning_r_to_t, multiple_couriers_planning_t_to_r, multiple_couriers_planning_incremental, multiple_couriers_planning_incremental_2]

compare_list_of_models(models, [f'../instances/inst0{i}.dat' for i in range(1, 7)] + [f'../instances/inst0{i}.dat' for i in range(8, 9)], search='Linear')

----------inst01.dat-----------
Model 0: (14, '0.2s'), total time: 0.9s
Model 1: (14, '0.2s'), total time: 0.7s
Model 2: (14, '0.1s'), total time: 0.7s
Model 3: (14, '0.2s'), total time: 0.8s
----------inst02.dat-----------
Model 0: (226, '6.8s'), total time: 15.7s
Model 1: (226, '7.3s'), total time: 16.3s
Model 2: (226, '3.0s'), total time: 11.2s
Model 3: (226, '15.8s'), total time: 23.3s
----------inst03.dat-----------
Model 0: (12, '0.3s'), total time: 2.5s
Model 1: (12, '0.3s'), total time: 1.5s
Model 2: (12, '0.4s'), total time: 3.1s
Model 3: (12, '0.4s'), total time: 2.3s
----------inst04.dat-----------
Model 0: (220, '7.0s'), total time: 22.5s
Model 1: (220, '2.5s'), total time: 16.9s
Model 2: (220, '2.9s'), total time: 16.9s
Model 3: (220, '0.6s'), total time: 14.7s
----------inst05.dat-----------
Model 0: (206, '0.0s'), total time: 0.3s
Model 1: (206, '0.0s'), total time: 0.3s
Model 2: (206, '0.1s'), total time: 0.3s
Model 3: (206, '0.1s'), total time: 0.3s
----------inst06.da

In [ ]:
test_model(multiple_couriers_planning_2, 'model1.txt', 15, display_solution=True)

In [ ]:
compare_models(multiple_couriers_planning_1, multiple_couriers_planning_2, 'model1', 'model2_NoEncodingTime', 8, display_solution=True)

# Unit testing area (to be removed)

In [ ]:
# # testing of conditional_sum_K_bin
# n = 4
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 4]
# digits = num_bits(max(e for e in alpha))
# alpha_bin = [int_to_bin(e, digits) for e in alpha]

# for comb in range(2**n):
#     s = Solver()
#     somma = 0
#     for i in range(n):
#         # print(comb, i, comb % (2**i))
#         if comb >= (2**i) and comb % (2**(i+1)) == 1:
#             print(comb, 2**i)
#             s.add(x[i])
#             somma += alpha[i]
#         else:
#             s.add(Not(x[i]))

#     delta = [Bool(f'delta_{j}') for j in range(8)]
#     s.add(conditional_sum_K_bin(x, alpha_bin, delta, 'ciao'))
#     s.check()
#     m = s.model()
#     print(f"{somma} ?= {bin_to_int([1 if m.evaluate(e) else 0 for e in delta])}")


# # print(s.check())
# # m = s.model()
# # print([m.evaluate(e) for e in x])
# # print([m.evaluate(e) for e in delta])

In [ ]:
# # Testing of LinearInteger_bin
# n = 4
# digits = 6
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 6]
# alpha_bin = [int_to_bin(e, digits) for e in alpha]
# beta = int_to_bin(5, digits)
# s = Solver()
# s.add(LinearInteger_bin(x, alpha_bin, beta, 'ciao'))
# s.add(Or(x))

# print(s.check())
# m = s.model()
# print([m.evaluate(e) for e in x])


Counting how many more 2-regular graphs are there w.r.t. 2-regular connected graphs (=> Hamiltonian cycles)

In [34]:
import math
import numpy as np

memo = [-1] * 270
memo[0] = 0
memo[1] = 0
memo[2] = 1
memo[3] = 2

def g(n):
    if memo[n] != -1:
        return memo[n]
    ans = 0
    for k in range(2, n-1):
        z = 1
        for i in range(1, k):
            z *= n-i
        z *= g(n-k)
        ans += z
    memo[n] = ans
    return memo[n]

In [41]:
print([g(x) for x in range(1, 20)])
print([math.factorial(x-1) for x in range(1, 20)])
print([g(x) / math.factorial(x-1) for x in range(1, 270)])

[0, 1, 2, 3, 20, 115, 810, 6475, 58280, 582795, 6410750, 76928995, 1000076940, 14001077155, 210016157330, 3360258517275, 57124394793680, 1028239106286235, 19536543019438470]
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800, 39916800, 479001600, 6227020800, 87178291200, 1307674368000, 20922789888000, 355687428096000, 6402373705728000]
[0.0, 1.0, 1.0, 0.5, 0.8333333333333334, 0.9583333333333334, 1.125, 1.2847222222222223, 1.445436507936508, 1.6060267857142858, 1.7666308421516754, 1.9272335207231042, 2.0878363245550746, 2.2484391179486667, 2.4090419121452107, 2.569644706284401, 2.7302475004274154, 2.8908502945701904, 3.0514530887129796, 3.212055882855768, 3.3726586769985563, 3.5332614711413446, 3.693864265284133, 3.8544670594269212, 4.01506985356971, 4.175672647712498, 4.336275441855286, 4.496878235998075, 4.657481030140864, 4.818083824283652, 4.9786866184264404, 5.139289412569228, 5.299892206712017, 5.460495000854805, 5.621097794997594, 5.7817005891403825, 5.94230338328317, 6.1029